In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
import requests
from functions_evaluation import *
from io import StringIO
from datetime import timedelta, datetime
import warnings
warnings.filterwarnings('ignore')

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# Loading surveillance data for evaluation: last surveillance file
start_date = datetime(2023, 9, 9)
state = 'US'
github_repo = "cdcepi/FluSight-forecast-hub"
github_directory = "auxiliary-data/target-data-archive"
surveillance_file = "target-hospital-admissions_2024-04-27.csv"
horizon_to_start = 5
df_surv = loading_surveillance_eval(start_date, github_repo, github_directory, surveillance_file, state, horizon_to_start)
df_surv.head()

,date,location,location_name,hospitalizations,weekly_rate,horizon
1582,2023-10-07,US,US,1116,0.335942,5
1529,2023-10-14,US,US,1218,0.366647,6
1476,2023-10-21,US,US,1474,0.443709,7
1423,2023-10-28,US,US,1610,0.484648,8
1370,2023-11-04,US,US,1974,0.594220,9


In [3]:
adaptive_ensemble2_path = "../output_data/adaptive_ensemble2_forecasts"
url_flusight = "https://raw.githubusercontent.com/cdcepi/Flusight-ensemble/main/Archive_2324/FluSight-ensemble/"   
url_baseline = "https://raw.githubusercontent.com/cdcepi/Flusight-baseline/main/Archive_2324/FluSight Baseline Forecasts/"
surv_lookup = df_surv[['date', 'hospitalizations']].drop_duplicates()
k_values = [0.05, 0.15,  0.25, 0.5, 0.75]
horizon_to_date = df_surv.set_index('horizon')['date'].to_dict()

In [7]:
# wis, mae, coverage for adaptive ensemble
dict_wis_perc_horizon = {}
dict_ae_perc_horizon = {}
dict_cov_perc_horizon = {}
# wis, mae, coverage for baseline
dict_wis_perc_horizon_bas = {}
dict_ae_perc_horizon_bas = {}
dict_cov_perc_horizon_bas = {}
# wis, mae, coverage for flusight ensemble
dict_wis_perc_horizon_flusight = {}
dict_ae_perc_horizon_flusight = {}
dict_cov_perc_horizon_flusight = {}

horizons = [0, 1, 2, 3]
unique_rounds = df_surv['horizon'].unique()[1:-3]
unique_dates = df_surv['date'].sort_values().unique()[1:-3]
alpha_vals = [0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.975]
alphas = ["{:.2f}".format((1 - a) / 2) for a in alpha_vals]
upper_alphas = ["{:.2f}".format(1 - (1 - a) / 2) for a in alpha_vals]

for h in horizons:
    for d in [dict_wis_perc_horizon, dict_ae_perc_horizon, dict_cov_perc_horizon,
              dict_wis_perc_horizon_bas, dict_ae_perc_horizon_bas, dict_cov_perc_horizon_bas,
              dict_wis_perc_horizon_flusight, dict_ae_perc_horizon_flusight, dict_cov_perc_horizon_flusight]:
        d[h] = {}

for k in k_values:
    k_perc = int(k * 100)
    dict_wis_adaptive_all_h = {h: {} for h in horizons}
    dict_ae_adaptive_all_h = {h: {} for h in horizons}
    dict_cov_adaptive_all_h = {h: {} for h in horizons}

    dict_wis_bas_all_h = {h: {} for h in horizons}
    dict_ae_bas_all_h = {h: {} for h in horizons}
    dict_cov_bas_all_h = {h: {} for h in horizons}

    dict_wis_flu_all_h = {h: {} for h in horizons}
    dict_ae_flu_all_h = {h: {} for h in horizons}
    dict_cov_flu_all_h = {h: {} for h in horizons}

    for round in unique_rounds:
        date = df_surv[df_surv['horizon'] == round]['date'].unique()[0]
        day_str = pd.to_datetime(date).strftime('%Y-%m-%d')

        # ----------------- ADAPTIVE ENSEMBLE -----------------
        path_adaptive = f"{adaptive_ensemble2_path}/{day_str}_{k}.csv"
        df_ensemble = import_ensemble2_forecast(path_adaptive, surv_lookup, horizon_to_date)
        dict_wis_h, dict_ae_h = compute_dict_WIS_AE_forecast(df_ensemble)

        df_ensemble['output_type_id_str'] = df_ensemble['output_type_id'].apply(lambda x: "{:.2f}".format(x))
        for h in horizons:
            dict_wis_adaptive_all_h[h][round] = dict_wis_h[h]
            dict_ae_adaptive_all_h[h][round] = dict_ae_h[h]

            coverages = []
            for i, alpha in enumerate(alphas):
                lower, upper = alpha, upper_alphas[i]
                df_q = df_ensemble[(df_ensemble.horizon == h) &
                                   (df_ensemble.output_type_id_str.isin([lower, upper]))]
                if not df_q.empty:
                    df_cov = df_q.groupby(["horizon", "date"], as_index=False).apply(coverage, lower, upper)
                    coverage_val = df_cov[None].mean() if df_cov.shape[0] > 0 else np.nan
                    coverages.append(coverage_val)
                else:
                    coverages.append(np.nan)
            dict_cov_adaptive_all_h[h][round] = coverages

        # ----------------- FLUSIGHT BASELINE -----------------
        df_baseline = import_baseline(day_str)
        df_baseline = df_baseline.rename(columns={"target_end_date": "date"})
        df_baseline = df_baseline[df_baseline.location == 'US']
        df_baseline['date'] = pd.to_datetime(df_baseline['date'])
        df_baseline['output_type_id_str'] = df_baseline['output_type_id'].apply(lambda x: "{:.2f}".format(x))

        df_bas = pd.merge(df_baseline, df_surv[['date', 'hospitalizations']], how='left', on="date")
        dict_wis_h, dict_ae_h = compute_dict_WIS_AE_forecast(df_bas)

        for h in horizons:
            dict_wis_bas_all_h[h][round] = dict_wis_h[h]
            dict_ae_bas_all_h[h][round] = dict_ae_h[h]

            coverages = []
            for i, alpha in enumerate(alphas):
                lower, upper = alpha, upper_alphas[i]
                df_q = df_bas[(df_bas.horizon == h) &
                              (df_bas.output_type_id_str.isin([lower, upper]))]
                if not df_q.empty:
                    df_cov = df_q.groupby(["horizon", "date"], as_index=False).apply(coverage, lower, upper)
                    coverage_val = df_cov[None].mean() if df_cov.shape[0] > 0 else np.nan
                    coverages.append(coverage_val)
                else:
                    coverages.append(np.nan)
            dict_cov_bas_all_h[h][round] = coverages

        # ----------------- FLUSIGHT ENSEMBLE -----------------
        df_flusight = import_ens_flusight(day_str)
        if not df_flusight.empty:
            df_flusight = df_flusight.rename(columns={"target_end_date": "date"})
            df_flusight = df_flusight[(df_flusight.location == 'US') & (df_flusight.output_type == 'quantile')]
            df_flusight['date'] = pd.to_datetime(df_flusight['date'])
            df_flusight['output_type_id_str'] = df_flusight['output_type_id'].apply(lambda x: "{:.2f}".format(float(x)))

            df_flu = pd.merge(df_flusight, df_surv[['date', 'hospitalizations']], how='left', on="date")
            dict_wis_h, dict_ae_h = compute_dict_WIS_AE_forecast(df_flu)

            for h in horizons:
                dict_wis_flu_all_h[h][round] = dict_wis_h[h]
                dict_ae_flu_all_h[h][round] = dict_ae_h[h]

                coverages = []
                for i, alpha in enumerate(alphas):
                    lower, upper = alpha, upper_alphas[i]
                    df_q = df_flu[(df_flu.horizon == h) &
                                  (df_flu.output_type_id_str.isin([lower, upper]))]
                    if not df_q.empty:
                        df_cov = df_q.groupby(["horizon", "date"], as_index=False).apply(coverage, lower, upper)
                        coverage_val = df_cov[None].mean() if df_cov.shape[0] > 0 else np.nan
                        coverages.append(coverage_val)
                    else:
                        coverages.append(np.nan)
                dict_cov_flu_all_h[h][round] = coverages
        else:
            for h in horizons:
                dict_wis_flu_all_h[h][round] = np.nan
                dict_ae_flu_all_h[h][round] = np.nan
                dict_cov_flu_all_h[h][round] = [np.nan] * len(alphas)

    for h in horizons:
        dict_wis_perc_horizon[h][k_perc] = dict_wis_adaptive_all_h[h]
        dict_ae_perc_horizon[h][k_perc] = dict_ae_adaptive_all_h[h]
        dict_cov_perc_horizon[h][k_perc] = dict_cov_adaptive_all_h[h]

        dict_wis_perc_horizon_bas[h][k_perc] = dict_wis_bas_all_h[h]
        dict_ae_perc_horizon_bas[h][k_perc] = dict_ae_bas_all_h[h]
        dict_cov_perc_horizon_bas[h][k_perc] = dict_cov_bas_all_h[h]

        dict_wis_perc_horizon_flusight[h][k_perc] = dict_wis_flu_all_h[h]
        dict_ae_perc_horizon_flusight[h][k_perc] = dict_ae_flu_all_h[h]
        dict_cov_perc_horizon_flusight[h][k_perc] = dict_cov_flu_all_h[h]


Failed to fetch file. Status code: 404, Message: 404: Not Found
Failed to fetch file. Status code: 404, Message: 404: Not Found
Failed to fetch file. Status code: 404, Message: 404: Not Found
Failed to fetch file. Status code: 404, Message: 404: Not Found
Failed to fetch file. Status code: 404, Message: 404: Not Found


In [6]:
dict_wis_perc_horizon = {}
dict_ae_perc_horizon = {}
dict_wis_perc_horizon_bas = {}
dict_ae_perc_horizon_bas = {}
dict_wis_perc_horizon_flusight = {}
dict_ae_perc_horizon_flusight = {}

horizons = [0, 1, 2, 3]
unique_rounds = df_surv['horizon'].unique()[1:-3]
unique_dates = df_surv['date'].sort_values().unique()[1:-3]

for h in horizons:
    dict_wis_perc_horizon[h] = {}
    dict_ae_perc_horizon[h] = {}
    dict_wis_perc_horizon_bas[h] = {}
    dict_ae_perc_horizon_bas[h] = {}
    dict_wis_perc_horizon_flusight[h] = {}
    dict_ae_perc_horizon_flusight[h] = {}

for k in k_values:
    k_perc = int(k * 100)
    # Init containers per k_perc across all horizons
    dict_wis_adaptive_all_h = {h: {} for h in horizons}
    dict_ae_adaptive_all_h = {h: {} for h in horizons}
    dict_wis_bas_all_h = {h: {} for h in horizons}
    dict_ae_bas_all_h = {h: {} for h in horizons}
    dict_wis_flu_all_h = {h: {} for h in horizons}
    dict_ae_flu_all_h = {h: {} for h in horizons}

    dict_cov_all_h = {h: {} for h in horizons}
    dict_cov_bas_all_h = {h: {} for h in horizons}
    dict_cov_flu_all_h = {h: {} for h in horizons}

    for round in unique_rounds:
        date = df_surv[df_surv['horizon'] == round]['date'].unique()[0]

        # ----------------------------- Adaptive Ensemble -----------------------------
        full_path_adaptive = f"{adaptive_ensemble2_path}/{date.strftime('%Y-%m-%d')}_{k}.csv"
        df_ensemble = import_ensemble2_forecast(full_path_adaptive, surv_lookup, horizon_to_date)
        dict_wis_horizon, dict_ae_horizon = compute_dict_WIS_AE_forecast(df_ensemble)
        for h in horizons:
            dict_wis_adaptive_all_h[h][round] = dict_wis_horizon[h]
            dict_ae_adaptive_all_h[h][round] = dict_ae_horizon[h]

        # ----------------------------- Baseline -----------------------------
        day_str = pd.to_datetime(date).strftime('%Y-%m-%d')
        df_baseline = import_baseline(day_str)
        df_baseline = df_baseline.rename(columns={"target_end_date": "date"})
        df_baseline = df_baseline[df_baseline.location == 'US']
        df_baseline['date'] = pd.to_datetime(df_baseline['date'])

        df_bas = pd.merge(df_baseline, df_surv[['date', 'hospitalizations']], how='left', on="date")
        dict_wis_horizon, dict_ae_horizon = compute_dict_WIS_AE_forecast(df_bas)

        for h in horizons:
            dict_wis_bas_all_h[h][round] = dict_wis_horizon[h]
            dict_ae_bas_all_h[h][round] = dict_ae_horizon[h]

        # ----------------------------- FluSight Ensemble -----------------------------
        df_flusight = import_ens_flusight(day_str)
        if df_flusight.empty:
            for h in horizons:
                dict_wis_flu_all_h[h][round] = np.nan
                dict_ae_flu_all_h[h][round] = np.nan
        else:
            df_flusight = df_flusight.rename(columns={"target_end_date": "date"})
            df_flusight = df_flusight[(df_flusight.location == 'US') & (df_flusight.output_type == 'quantile')]
            df_flusight['date'] = pd.to_datetime(df_flusight['date'])

            df_flu = pd.merge(df_flusight, df_surv[['date', 'hospitalizations']], how='left', on="date")
            dict_wis_horizon, dict_ae_horizon = compute_dict_WIS_AE_forecast(df_flu)

            for h in horizons:
                dict_wis_flu_all_h[h][round] = dict_wis_horizon[h]
                dict_ae_flu_all_h[h][round] = dict_ae_horizon[h]

    for h in horizons:
        dict_wis_perc_horizon[h][k_perc] = dict_wis_adaptive_all_h[h]
        dict_ae_perc_horizon[h][k_perc] = dict_ae_adaptive_all_h[h]
        dict_wis_perc_horizon_bas[h][k_perc] = dict_wis_bas_all_h[h]
        dict_ae_perc_horizon_bas[h][k_perc] = dict_ae_bas_all_h[h]
        dict_wis_perc_horizon_flusight[h][k_perc] = dict_wis_flu_all_h[h]
        dict_ae_perc_horizon_flusight[h][k_perc] = dict_ae_flu_all_h[h]

KeyboardInterrupt: 

In [6]:
# WIS performance - Adaptive Ensemble against Baseline and FluSight
rows = []
for horizon, k_dict in dict_wis_perc_horizon.items():
    for k_perc, round_dict in k_dict.items():
        for round_idx, value in round_dict.items():
            # get baseline 
            baseline_value = dict_wis_perc_horizon_bas[horizon][k_perc][round_idx]
            # get flu sight
            flu_value = dict_wis_perc_horizon_flusight[horizon][k_perc][round_idx]
            rows.append({
                "horizon": horizon,
                "k_perc": k_perc,
                "round": round_idx,
                "wis_adaptive": value,
                "wis_baseline": baseline_value,
                "wis_flusight": flu_value,
                "wis_adaptive_baseline": value/baseline_value,
                "wis_adaptive_flusight": value/flu_value
            })
df_wis_k_h = pd.DataFrame(rows)
df_wis_k_h.to_csv("../output_data/performance_forecasting/wis_adaptive_ensemble2.csv", index=False)
df_wis_k_h

,horizon,k_perc,round,wis_adaptive,wis_baseline,wis_flusight,wis_adaptive_baseline,wis_adaptive_flusight
0,0,5,6,102.202759,322.999838,162.264737,0.316417,0.629852
1,0,5,7,96.968091,401.427733,197.621614,0.241558,0.490676
2,0,5,8,109.253361,422.483221,162.146175,0.258598,0.673795
3,0,5,9,169.352045,350.566391,221.685064,0.483081,0.763931
4,0,5,10,372.051193,600.136826,370.419915,0.619944,1.004404
...,...,...,...,...,...,...,...,...
515,3,75,27,1927.975715,3556.884735,1496.327088,0.542041,1.288472
516,3,75,28,1859.319832,2983.781311,1354.812458,0.623142,1.372382
517,3,75,29,1502.509492,2413.072282,1002.187332,0.622654,1.499230
518,3,75,30,1082.311924,2223.462224,896.339511,0.486769,1.207480


In [7]:
# MAE performance - Adaptive Ensemble against Baseline and FluSight
rows = []
for horizon, k_dict in dict_ae_perc_horizon.items():
    for k_perc, round_dict in k_dict.items():
        for round_idx, value in round_dict.items():
            # get baseline 
            baseline_value = dict_ae_perc_horizon_bas[horizon][k_perc][round_idx]
            # get flu sight
            flu_value = dict_ae_perc_horizon_flusight[horizon][k_perc][round_idx]
            rows.append({
                "horizon": horizon,
                "k_perc": k_perc,
                "round": round_idx,
                "mae_adaptive": value,
                "mae_baseline": baseline_value,
                "mae_flusight": flu_value,
                "mae_adaptive_baseline": value/baseline_value,
                "mae_adaptive_flusight": value/flu_value
            })
df_mae_k_h = pd.DataFrame(rows)
df_mae_k_h.to_csv("../output_data/performance_forecasting/mae_adaptive_ensemble2.csv", index=False)
df_mae_k_h

,horizon,k_perc,round,mae_adaptive,mae_baseline,mae_flusight,mae_adaptive_baseline,mae_adaptive_flusight
0,0,5,6,0.024706,0.076355,0.025385,0.323571,0.973258
1,0,5,7,0.073390,0.215739,0.137274,0.340181,0.534628
2,0,5,8,0.082393,0.221739,0.063848,0.371577,1.290448
3,0,5,9,0.119182,0.182877,0.122340,0.651706,0.974185
4,0,5,10,0.208058,0.256030,0.195673,0.812633,1.063293
...,...,...,...,...,...,...,...,...
515,3,75,27,0.447897,0.905601,0.353054,0.494585,1.268636
516,3,75,28,0.567211,0.839224,0.340335,0.675875,1.666624
517,3,75,29,0.566433,0.839272,0.170589,0.674910,3.320445
518,3,75,30,0.516100,1.069580,0.333217,0.482526,1.548842


In [9]:
rows = []
alpha_vals = [0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.975]
alphas = ["{:.2f}".format((1 - a) / 2) for a in alpha_vals]
upper_alphas = ["{:.2f}".format(1 - (1 - a) / 2) for a in alpha_vals]
alpha_labels = [f"{a}-{b}" for a, b in zip(alphas, upper_alphas)]

for horizon, k_dict in dict_cov_perc_horizon.items():
    for k_perc, round_dict in k_dict.items():
        for round_idx, adaptive_list in round_dict.items():
            baseline_list = dict_cov_perc_horizon_bas[horizon][k_perc].get(round_idx, [np.nan]*len(alpha_labels))
            flusight_list = dict_cov_perc_horizon_flusight[horizon][k_perc].get(round_idx, [np.nan]*len(alpha_labels))
            
            row = {
                "horizon": horizon,
                "k_perc": k_perc,
                "round": round_idx
            }
            for i, label in enumerate(alpha_labels):
                row[f"cov_adaptive_{label}"] = adaptive_list[i] if i < len(adaptive_list) else np.nan
                row[f"cov_baseline_{label}"] = baseline_list[i] if i < len(baseline_list) else np.nan
                row[f"cov_flusight_{label}"] = flusight_list[i] if i < len(flusight_list) else np.nan

            rows.append(row)

df_cov_k_h = pd.DataFrame(rows)
df_cov_k_h.to_csv("../output_data/performance_forecasting/coverage_adaptive_ensemble2.csv", index=False)
df_cov_k_h

,horizon,k_perc,round,cov_adaptive_0.45-0.55,cov_baseline_0.45-0.55,cov_flusight_0.45-0.55,cov_adaptive_0.40-0.60,cov_baseline_0.40-0.60,cov_flusight_0.40-0.60,cov_adaptive_0.35-0.65,...,cov_flusight_0.10-0.90,cov_adaptive_0.05-0.95,cov_baseline_0.05-0.95,cov_flusight_0.05-0.95,cov_adaptive_0.03-0.97,cov_baseline_0.03-0.97,cov_flusight_0.03-0.97,cov_adaptive_0.01-0.99,cov_baseline_0.01-0.99,cov_flusight_0.01-0.99
0,0,5,6,0.0,0.0,1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0,5,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0,5,8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0,5,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0,5,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,3,75,27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
516,3,75,28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
517,3,75,29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
518,3,75,30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
